<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

TensorFlow 2.x selected.


'2.0.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [2]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [0]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [5]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [6]:
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-3.043519,-3.043519
1,-1.995282,-1.995282
2,-1.991833,-1.991833
3,-1.835947,-1.835947
4,-1.792003,-1.792003


In [7]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [8]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-3.043519,0.045498
1,-1.995282,0.119699
2,-1.991833,0.120063
3,-1.835947,0.137531
4,-1.792003,0.142827


In [9]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [10]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-3.043519,0.0
1,-1.995282,0.0
2,-1.991833,0.0
3,-1.835947,0.0
4,-1.792003,0.0


In [11]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [12]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-3.043519,-0.995466
1,-1.995282,-0.963693
2,-1.991833,-0.963446
3,-1.835947,-0.950405
4,-1.792003,-0.945972


In [13]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [14]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [0]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [0]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [16]:
data = np.random.randn(1000, 150)
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [17]:
data[:3]

array([[ 1.13213781e-01,  1.13334623e+00, -1.47733450e+00,
        -1.64283435e+00, -3.37696542e-01, -4.96299921e-01,
         1.28196024e+00, -3.11399917e-01, -2.05001741e+00,
        -1.76701750e+00,  4.72642663e-02,  6.07742114e-01,
        -4.77195617e-01,  1.76008529e+00,  2.07281421e+00,
        -8.68255080e-01,  3.38672122e-01, -3.73632133e-01,
         6.60135938e-01, -1.08493514e+00, -1.45003690e+00,
        -1.14411382e-01,  1.39901377e+00, -8.79934513e-01,
        -7.01294981e-01, -1.31591441e+00,  2.02749851e+00,
        -1.62197213e+00, -3.11525032e-01, -1.84663977e-01,
        -5.64537845e-01, -3.55172905e-01, -5.23102739e-01,
         9.55654023e-01, -1.37066020e+00,  1.81870272e+00,
         5.83585220e-01, -3.23434867e-01, -6.87775926e-01,
         9.96908049e-01,  8.13596098e-01, -5.32404088e-01,
        -1.31850109e+00,  3.37938320e-01, -1.34072823e+00,
         2.07065174e+00, -8.59703960e-01, -6.72337926e-01,
        -1.20225214e-01, -4.40443186e-01, -1.36830737e+0

In [18]:
labels[:10]

array([[1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0]])

In [19]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Train on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.8066 - accuracy: 0.4950
Epoch 2/20
1000/1000 [==============================] - 0s 65us/sample - loss: 0.7048 - accuracy: 0.5700
Epoch 3/20
1000/1000 [==============================] - 0s 61us/sample - loss: 0.6511 - accuracy: 0.6250
Epoch 4/20
1000/1000 [==============================] - 0s 50us/sample - loss: 0.6075 - accuracy: 0.6810
Epoch 5/20
1000/1000 [==============================] - 0s 64us/sample - loss: 0.5722 - accuracy: 0.7220
Epoch 6/20
1000/1000 [==============================] - 0s 67us/sample - loss: 0.5398 - accuracy: 0.7640
Epoch 7/20
1000/1000 [==============================] - 0s 58us/sample - loss: 0.5125 - accuracy: 0.7780
Epoch 8/20
1000/1000 [==============================] - 0s 63us/sample - loss: 0.4871 - accuracy: 0.7970
Epoch 9/20
1000/1000 [==============================] - 0s 54us/sample - loss: 0.4632 - accuracy: 0.8200
Epoch 10/20
1000/1000 [===========

In [20]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=30)

Train on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 504us/sample - loss: 0.8355 - accuracy: 0.5140
Epoch 2/20
1000/1000 [==============================] - 0s 71us/sample - loss: 0.7114 - accuracy: 0.5630
Epoch 3/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.6549 - accuracy: 0.6020
Epoch 4/20
1000/1000 [==============================] - 0s 74us/sample - loss: 0.6130 - accuracy: 0.6460
Epoch 5/20
1000/1000 [==============================] - 0s 73us/sample - loss: 0.5789 - accuracy: 0.6900
Epoch 6/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.5491 - accuracy: 0.7310
Epoch 7/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.5218 - accuracy: 0.7550
Epoch 8/20
1000/1000 [==============================] - 0s 75us/sample - loss: 0.4976 - accuracy: 0.7780
Epoch 9/20
1000/1000 [==============================] - 0s 65us/sample - loss: 0.4733 - accuracy: 0.8080
Epoch 10/20
1000/1000 [=========

In [21]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s 979us/sample - loss: 0.8053 - accuracy: 0.5188 - val_loss: 0.7709 - val_accuracy: 0.4850
Epoch 2/20
800/800 [==============================] - 0s 77us/sample - loss: 0.7056 - accuracy: 0.5688 - val_loss: 0.7608 - val_accuracy: 0.4900
Epoch 3/20
800/800 [==============================] - 0s 83us/sample - loss: 0.6510 - accuracy: 0.6200 - val_loss: 0.7563 - val_accuracy: 0.5100
Epoch 4/20
800/800 [==============================] - 0s 90us/sample - loss: 0.6073 - accuracy: 0.6650 - val_loss: 0.7554 - val_accuracy: 0.5450
Epoch 5/20
800/800 [==============================] - 0s 82us/sample - loss: 0.5714 - accuracy: 0.7025 - val_loss: 0.7583 - val_accuracy: 0.5500
Epoch 6/20
800/800 [==============================] - 0s 80us/sample - loss: 0.5403 - accuracy: 0.7325 - val_loss: 0.7610 - val_accuracy: 0.5400
Epoch 7/20
800/800 [==============================] - 0s 80us/sample - loss: 0.5121

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [24]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [25]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s 904us/sample - loss: 0.8627 - accuracy: 0.4988 - val_loss: 0.8060 - val_accuracy: 0.5450
Epoch 2/20
800/800 [==============================] - 0s 79us/sample - loss: 0.7264 - accuracy: 0.5663 - val_loss: 0.7739 - val_accuracy: 0.5650
Epoch 3/20
800/800 [==============================] - 0s 75us/sample - loss: 0.6564 - accuracy: 0.6150 - val_loss: 0.7584 - val_accuracy: 0.5650
Epoch 4/20
800/800 [==============================] - 0s 80us/sample - loss: 0.6053 - accuracy: 0.6750 - val_loss: 0.7516 - val_accuracy: 0.5700
Epoch 5/20
800/800 [==============================] - 0s 78us/sample - loss: 0.5654 - accuracy: 0.7075 - val_loss: 0.7514 - val_accuracy: 0.5700
Epoch 6/20
800/800 [==============================] - 0s 93us/sample - loss: 0.5313 - accuracy: 0.7538 - val_loss: 0.7549 - val_accuracy: 0.5950
Epoch 7/20
800/800 [==============================] - 0s 87us/sample - loss: 0.5012

In [0]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [27]:
model.predict(test_data)

array([[0.7205988 ],
       [0.9648355 ],
       [0.05887491],
       [0.43474725],
       [0.17025241]], dtype=float32)

In [28]:
model.predict_proba(test_data)

array([[0.7205988 ],
       [0.9648355 ],
       [0.05887491],
       [0.43474725],
       [0.17025241]], dtype=float32)

In [29]:
model.predict_classes(test_data)

array([[1],
       [1],
       [0],
       [0],
       [0]], dtype=int32)

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [0]:
data = np.random.random((1000, 150))
labels = np.random.randint(10, size=(1000, 1))

In [31]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [32]:
data[:3]

array([[7.29627635e-01, 8.62521376e-01, 5.63809956e-01, 1.54166381e-01,
        3.70730181e-01, 6.10089486e-01, 7.74374499e-01, 5.61109957e-01,
        8.17466080e-01, 2.42193103e-01, 7.53253516e-01, 5.66456363e-02,
        3.79409962e-01, 6.54154010e-01, 7.82308693e-01, 2.90854633e-01,
        1.11945416e-02, 2.64953899e-02, 4.10102070e-01, 4.74431076e-01,
        1.07910501e-01, 4.58667130e-01, 4.43274373e-02, 7.79824111e-01,
        5.41952852e-01, 5.30996524e-01, 5.00869014e-01, 6.62376550e-01,
        6.38701216e-01, 5.99692135e-01, 9.23143429e-01, 3.25124649e-01,
        8.18399850e-01, 7.44213954e-01, 8.15133320e-01, 4.15864081e-02,
        4.32821197e-01, 2.94940677e-01, 2.24524351e-01, 8.92774388e-01,
        2.42985440e-02, 8.10418544e-01, 6.02631336e-01, 1.00811505e-01,
        6.56807993e-01, 3.93307149e-01, 4.32772410e-01, 6.20595873e-01,
        5.43492515e-01, 7.89735153e-01, 5.96972647e-01, 4.61710865e-01,
        3.74244480e-01, 2.04553053e-01, 6.55018074e-01, 1.516844

In [33]:
labels[:10]

array([[3],
       [6],
       [2],
       [3],
       [6],
       [4],
       [4],
       [2],
       [0],
       [5]])

In [34]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [35]:
labels[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [36]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 645us/sample - loss: 2.3556 - accuracy: 0.1050 - val_loss: 2.3490 - val_accuracy: 0.0750
Epoch 2/30
800/800 [==============================] - 0s 75us/sample - loss: 2.3041 - accuracy: 0.1163 - val_loss: 2.3348 - val_accuracy: 0.0700
Epoch 3/30
800/800 [==============================] - 0s 77us/sample - loss: 2.2927 - accuracy: 0.1213 - val_loss: 2.3311 - val_accuracy: 0.0850
Epoch 4/30
800/800 [==============================] - 0s 75us/sample - loss: 2.2827 - accuracy: 0.1262 - val_loss: 2.3372 - val_accuracy: 0.0850
Epoch 5/30
800/800 [==============================] - 0s 77us/sample - loss: 2.2714 - accuracy: 0.1475 - val_loss: 2.3350 - val_accuracy: 0.0850
Epoch 6/30
800/800 [==============================] - 0s 78us/sample - loss: 2.2611 - accuracy: 0.1713 - val_loss: 2.3264 - val_accuracy: 0.0900
Epoch 7/30
800/800 [==============================] - 0s 78us/sample - loss: 2.2535

In [37]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[0.08983808, 0.11159482, 0.1492207 , 0.09316637, 0.11945623,
        0.09258357, 0.0720282 , 0.06992816, 0.13106145, 0.07112241],
       [0.08942046, 0.10283626, 0.18557036, 0.05366269, 0.22781137,
        0.03010176, 0.06015954, 0.06327941, 0.06009176, 0.12706645],
       [0.10976189, 0.12452849, 0.06198036, 0.1371533 , 0.07977825,
        0.13205202, 0.08140872, 0.07753852, 0.07799304, 0.11780547],
       [0.05390581, 0.09509765, 0.05877729, 0.15198168, 0.14855085,
        0.03478269, 0.07228796, 0.10269187, 0.15391472, 0.12800953],
       [0.09683111, 0.11166308, 0.12938194, 0.0463362 , 0.14714347,
        0.06077057, 0.04766032, 0.06275054, 0.08363295, 0.2138298 ],
       [0.12804571, 0.08242729, 0.09793483, 0.11848731, 0.11091572,
        0.16548876, 0.0669103 , 0.07752561, 0.05154081, 0.10072374],
       [0.1094671 , 0.08647396, 0.09840713, 0.12729783, 0.08862638,
        0.09885453, 0.10153055, 0.08075281, 0.09815591, 0.11043376],
       [0.08532665, 0.09715191, 0.1484507

In [38]:
model.predict_classes(test_data)

array([2, 4, 3, 8, 9, 5, 3, 4, 3, 2])

### <a name='a6'></a> Przykład - regresja

In [0]:
data = np.random.random((1000, 150))
labels = 50 * np.random.random(1000)

In [40]:
data[:3]

array([[0.58960954, 0.13243576, 0.93943898, 0.40004257, 0.32280289,
        0.5722313 , 0.9073174 , 0.05766878, 0.79989911, 0.57997995,
        0.64221612, 0.82995811, 0.19042491, 0.38901094, 0.15819404,
        0.3175369 , 0.42458804, 0.74393206, 0.88612055, 0.58409325,
        0.96100708, 0.96303939, 0.23650876, 0.94119992, 0.59852204,
        0.10331191, 0.24064297, 0.49557905, 0.5794331 , 0.05596316,
        0.07132605, 0.83778146, 0.91188186, 0.75986717, 0.66353842,
        0.32291479, 0.03063383, 0.89332242, 0.60826858, 0.95087787,
        0.74390908, 0.44394929, 0.04739086, 0.57713395, 0.3310456 ,
        0.3187024 , 0.57310303, 0.02291328, 0.23263295, 0.48508289,
        0.99762691, 0.46855278, 0.35847907, 0.0393143 , 0.47897298,
        0.59401619, 0.89002729, 0.3034738 , 0.33718665, 0.78801235,
        0.8085318 , 0.70728711, 0.55510417, 0.18523857, 0.48092329,
        0.59647718, 0.27824617, 0.0198724 , 0.89486556, 0.63120354,
        0.66668975, 0.78255413, 0.12671226, 0.86

In [41]:
labels[:10]

array([15.1173711 , 17.67429876, 17.94821655, 44.47101798, 17.40075161,
       25.79769984, 29.71380237,  4.24483831, 46.75677128, 38.83994321])

In [42]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 0s 485us/sample - loss: 588.6155 - val_loss: 482.1194
Epoch 2/30
800/800 [==============================] - 0s 72us/sample - loss: 319.5847 - val_loss: 279.7593
Epoch 3/30
800/800 [==============================] - 0s 74us/sample - loss: 217.8644 - val_loss: 228.9637
Epoch 4/30
800/800 [==============================] - 0s 71us/sample - loss: 202.5274 - val_loss: 224.9378
Epoch 5/30
800/800 [==============================] - 0s 69us/sample - loss: 201.7686 - val_loss: 226.3445
Epoch 6/30
800/800 [==============================] - 0s 70us/sample - loss: 201.4993 - val_loss: 220.6610
Epoch 7/30
800/800 [==============================] - 0s 70us/sample - loss: 200.7321 - val_loss: 221.6260
Epoch 8/30
800/800 [==============================] - 0s 75us/sample - loss: 200.6722 - val_loss: 222.9951
Epoch 9/30
800/800 [==============================] - 0s 81us/sample - loss: 200.2090 - val_loss:

In [43]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 964us/sample - loss: 21.1680 - mse: 636.3564 - val_loss: 19.3503 - val_mse: 550.3011
Epoch 2/30
800/800 [==============================] - 0s 82us/sample - loss: 16.0822 - mse: 385.6503 - val_loss: 15.2038 - val_mse: 341.7426
Epoch 3/30
800/800 [==============================] - 0s 74us/sample - loss: 13.3585 - mse: 251.9436 - val_loss: 13.0714 - val_mse: 246.4524
Epoch 4/30
800/800 [==============================] - 0s 73us/sample - loss: 12.4629 - mse: 206.7149 - val_loss: 12.7177 - val_mse: 223.5341
Epoch 5/30
800/800 [==============================] - 0s 74us/sample - loss: 12.3362 - mse: 202.4299 - val_loss: 12.7078 - val_mse: 221.8473
Epoch 6/30
800/800 [==============================] - 0s 79us/sample - loss: 12.3174 - mse: 202.3679 - val_loss: 12.7101 - val_mse: 222.0762
Epoch 7/30
800/800 [==============================] - 0s 85us/sample - loss: 12.3100 - mse: 202.2263 - val_

In [44]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[24.40516 ],
       [27.921484],
       [23.890505],
       [23.812803],
       [26.02559 ],
       [24.233002],
       [23.821136],
       [22.760138],
       [27.158115],
       [26.908585]], dtype=float32)